# Challenge 2: Sentiment Analysis

In this challenge we will learn sentiment analysis and practice performing sentiment analysis on Twitter tweets. 

## Introduction

Sentiment analysis is to *systematically identify, extract, quantify, and study affective states and subjective information* based on texts ([reference](https://en.wikipedia.org/wiki/Sentiment_analysis)). In simple words, it's to understand whether a person is happy or unhappy in producing the piece of text. Why we (or rather, companies) care about sentiment in texts? It's because by understanding the sentiments in texts, we will be able to know if our customers are happy or unhappy about our products and services. If they are unhappy, the subsequent action is to figure out what have caused the unhappiness and make improvements.

Basic sentiment analysis only understands the *positive* or *negative* (sometimes *neutral* too) polarities of the sentiment. More advanced sentiment analysis will also consider dimensions such as agreement, subjectivity, confidence, irony, and so on. In this challenge we will conduct the basic positive vs negative sentiment analysis based on real Twitter tweets.

NLTK comes with a [sentiment analysis package](https://www.nltk.org/api/nltk.sentiment.html). This package is great for dummies to perform sentiment analysis because it requires only the textual data to make predictions. For example:

```python
>>> from nltk.sentiment.vader import SentimentIntensityAnalyzer
>>> txt = "Ironhack is a Global Tech School ranked num 2 worldwide.   Our mission is to help people transform their careers and join a thriving community of tech professionals that love what they do."
>>> analyzer = SentimentIntensityAnalyzer()
>>> analyzer.polarity_scores(txt)
{'neg': 0.0, 'neu': 0.741, 'pos': 0.259, 'compound': 0.8442}
```

In this challenge, however, you will not use NLTK's sentiment analysis package because in your Machine Learning training in the past 2 weeks you have learned how to make predictions more accurate than that. The [tweets data](https://www.kaggle.com/kazanova/sentiment140) we will be using today are already coded for the positive/negative sentiment. You will be able to use the Naïve Bayes classifier you learned in the lesson to predict the sentiment of tweets based on the labels.

## Conducting Sentiment Analysis

### Loading and Exploring Data

The dataset we'll be using today is located on Kaggle (https://www.kaggle.com/kazanova/sentiment140). Once you have downloaded and imported the dataset, it you will need to define the columns names: df.columns = ['target','id','date','flag','user','text']

*Notes:* 

* The dataset is huuuuge (1.6m tweets). When you develop your data analysis codes, you can sample a subset of the data (e.g. 20k records) so that you will save a lot of time when you test your codes.

In [2]:
# your code here

import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [3]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('tagsets')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package tagsets to /root/nltk_data...
[nltk_data]   Unzipping help/tagsets.zip.


True

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
#Loading dataset, column names are as follows: ['target','id','date','flag','user','text']
DATASET_COLUMNS  = ["target", "id", "date", "flag", "user", "text"]
DATASET_ENCODING = "ISO-8859-1"
tweets = pd.read_csv('/content/drive/MyDrive/ironhack_2022/training.1600000.processed.noemoticon.csv',
                      encoding=DATASET_ENCODING , names=DATASET_COLUMNS)

In [6]:
tweets.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [7]:
#Take a random subset of about 20,000 rows

tweets = tweets.sample(n=20000)
tweets.head()

,target,id,date,flag,user,text
1415761,4,2057281058,Sat Jun 06 12:42:33 PDT 2009,NO_QUERY,civillove,going to get ice cream with winchester bitches...
633099,0,2232888775,Thu Jun 18 20:32:08 PDT 2009,NO_QUERY,corinnevielle,@Pr33mo can't right now i don't have time.
94680,0,1771153960,Mon May 11 23:22:16 PDT 2009,NO_QUERY,Lisamanics,I still hurt. And I'm knackered. Can't believe...
1423583,4,2058724127,Sat Jun 06 15:30:22 PDT 2009,NO_QUERY,MatthewDConway,@awobben no only if u hook it up like that is...
409337,0,2059623388,Sat Jun 06 17:18:16 PDT 2009,NO_QUERY,Marsaliath,"I know I want to,but I know I can't..."


### Prepare Textual Data for Sentiment Analysis

Now, apply the functions you have written in Challenge 1 to your whole data set. These functions include:

* `clean_up()`

* `tokenize()`

* `stem_and_lemmatize()`

* `remove_stopwords()`

Create a new column called `text_processed` in the dataframe to contain the processed data. At the end, your `text_processed` column should contain lists of word tokens that are cleaned up. Your data should look like below:

![Processed Data](https://github.com/aaronpereira92/lab-nlp/blob/master/your-code/data-cleaning-results.png?raw=1)

In [8]:
# your code here

def clean_up(s):
    """
    Cleans up numbers, URLs, and special characters from a string.

    Args:
        s: The string to be cleaned up.

    Returns:
        A string that has been cleaned up.
    """
    s = re.sub(r'(http|https|ftp|ssh)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?', ' ' , s) 
    s = re.sub(r'\W+',' ',s) # Replace everything non-alpahnumeric by ' '
    s = re.sub(r'\d+',' ',s) # Replace one or more digits by  ' '
    s_new = s.lower()

    return s_new

def tokenize(s):
    """
    Tokenize a string.

    Args:
        s: String to be tokenized.

    Returns:
        A list of words as the result of tokenization.
    """
    s_new = s.split()
    return s_new

def stem_and_lemmatize(l):
    """
    Perform stemming and lemmatization on a list of words.

    Args:
        l: A list of strings.

    Returns:
        A list of strings after being stemmed and lemmatized.
    """
    from nltk.stem import PorterStemmer
    from nltk.stem import WordNetLemmatizer
    porter = PorterStemmer()
    lemmatizer = WordNetLemmatizer()

    stemmed= []
    lemmetized = []
    for x in l:
      stemmed.append(porter.stem(x))
    for x in stemmed:
      lemmetized.append(lemmatizer.lemmatize(x))
    
    return lemmetized

def remove_stopwords(l):
    """
    Remove English stopwords from a list of strings.

    Args:
        l: A list of strings.

    Returns:
        A list of strings after stop words are removed.
    """
    from nltk.corpus import stopwords

    stop_words = list(stopwords.words('english')) 
    stop_words
    #This would remove whte spaces from the stop words
    for i in range(len(stop_words)):
        stop_words[i] = re.sub(r"\s*'\s*\w*","",stop_words[i])
    
    mylistnew = [word for word in l if word not in stop_words]
    
    return mylistnew    

In [9]:
tweets['text_processed'] = tweets.apply(lambda row: clean_up(row['text']), axis =1)#cleans numbers.,special characters, urls
tweets['text_processed'] = tweets.apply(lambda row: tokenize(row['text_processed']), axis =1)
tweets['text_processed'] = tweets.apply(lambda row: stem_and_lemmatize(row['text_processed']), axis =1)
tweets['text_processed'] = tweets.apply(lambda row: remove_stopwords(row['text_processed']), axis =1)

tweets

,target,id,date,flag,user,text,text_processed
1415761,4,2057281058,Sat Jun 06 12:42:33 PDT 2009,NO_QUERY,civillove,going to get ice cream with winchester bitches...,"[go, get, ice, cream, winchest, bitch]"
633099,0,2232888775,Thu Jun 18 20:32:08 PDT 2009,NO_QUERY,corinnevielle,@Pr33mo can't right now i don't have time.,"[pr, mo, right, time]"
94680,0,1771153960,Mon May 11 23:22:16 PDT 2009,NO_QUERY,Lisamanics,I still hurt. And I'm knackered. Can't believe...,"[still, hurt, knacker, believ, onli, tuesday]"
1423583,4,2058724127,Sat Jun 06 15:30:22 PDT 2009,NO_QUERY,MatthewDConway,@awobben no only if u hook it up like that is...,"[awobben, onli, u, hook, like, phone, fix]"
409337,0,2059623388,Sat Jun 06 17:18:16 PDT 2009,NO_QUERY,Marsaliath,"I know I want to,but I know I can't...","[know, want, know]"
...,...,...,...,...,...,...,...
1241006,4,1993924713,Mon Jun 01 11:08:28 PDT 2009,NO_QUERY,Angel3284a,@Surfergal1 SOrry i was having issues with my ...,"[surferg, sorri, wa, issu, comput, technolog, ..."
1380416,4,2052239726,Sat Jun 06 00:05:04 PDT 2009,NO_QUERY,Kimberleyward,is having a girlie day shopping - then back to...,"[girli, day, shop, back, look, cupcak, recip, ..."
1041994,4,1957147628,Thu May 28 23:39:07 PDT 2009,NO_QUERY,aimizubouken,@d2zone Huhu..gonna give twitting a rest this ...,"[zone, huhu, gonna, give, twit, rest, thi, nex..."
581200,0,2214213594,Wed Jun 17 16:25:37 PDT 2009,NO_QUERY,XxmasonXx,first time i havent been there in 4 years,"[first, time, havent, year]"


### Creating Bag of Words

The purpose of this step is to create a [bag of words](https://en.wikipedia.org/wiki/Bag-of-words_model) from the processed data. The bag of words contains all the unique words in your whole text body (a.k.a. *corpus*) with the number of occurrence of each word. It will allow you to understand which words are the most important features across the whole corpus.

Also, you can imagine you will have a massive set of words. The less important words (i.e. those of very low number of occurrence) do not contribute much to the sentiment. Therefore, you only need to use the most important words to build your feature set in the next step. In our case, we will use the top 5,000 words with the highest frequency to build the features.

In the cell below, combine all the words in `text_processed` and calculate the frequency distribution of all words. A convenient library to calculate the term frequency distribution is NLTK's `FreqDist` class ([documentation](https://www.nltk.org/api/nltk.html#module-nltk.probability)). Then select the top 5,000 words from the frequency distribution.

In [10]:
# your code here
bag_of_words = ''

for x in tweets['text_processed']:
  bag_of_words+= ' '. join(x)

#corpus = nltk.sent_tokenize(reviews)
bag_of_words = tokenize(bag_of_words)

In [11]:
from nltk.probability import FreqDist

In [12]:
fdist = nltk.FreqDist(bag_of_words)

In [13]:
#Here we get a list of tupels with the 5000 most frequent words along with their counts 
freq_words = fdist.most_common(5000)

In [14]:
freq_words

[('go', 1356),
 ('get', 1247),
 ('wa', 1217),
 ('thi', 1078),
 ('day', 1050),
 ('like', 952),
 ('good', 883),
 ('quot', 848),
 ('love', 797),
 ('work', 695),
 ('u', 682),
 ('time', 607),
 ('amp', 598),
 ('know', 594),
 ('got', 591),
 ('see', 586),
 ('thank', 583),
 ('want', 568),
 ('today', 567),
 ('realli', 555),
 ('one', 554),
 ('back', 533),
 ('miss', 521),
 ('hope', 510),
 ('im', 507),
 ('think', 503),
 ('make', 484),
 ('still', 471),
 ('feel', 471),
 ('new', 465),
 ('look', 462),
 ('need', 459),
 ('much', 445),
 ('lol', 424),
 ('well', 417),
 ('ha', 411),
 ('night', 407),
 ('come', 393),
 ('oh', 385),
 ('home', 376),
 ('watch', 372),
 ('last', 358),
 ('great', 344),
 ('wait', 338),
 ('onli', 314),
 ('hi', 313),
 ('twitter', 311),
 ('wish', 309),
 ('veri', 307),
 ('sleep', 301),
 ('thing', 296),
 ('tri', 293),
 ('would', 288),
 ('morn', 283),
 ('bad', 279),
 ('tomorrow', 275),
 ('take', 274),
 ('follow', 268),
 ('sorri', 264),
 ('even', 262),
 ('way', 258),
 ('could', 257),
 ('fun'

### Building Features

Now let's build the features. Using the top 5,000 words, create a 2-dimensional matrix to record whether each of those words is contained in each document (tweet). Then you also have an output column to indicate whether the sentiment in each tweet is positive. For example, assuming your bag of words has 5 items (`['one', 'two', 'three', 'four', 'five']`) out of 4 documents (`['A', 'B', 'C', 'D']`), your feature set is essentially:

| Doc | one | two | three | four | five | is_positive |
|---|---|---|---|---|---|---|
| A | True | False | False | True | False | True |
| B | False | False | False | True | True | False |
| C | False | True | False | False | False | True |
| D | True | False | False | False | True | False|

However, because the `nltk.NaiveBayesClassifier.train` class we will use in the next step does not work with Pandas dataframe, the structure of your feature set should be converted to the Python list looking like below:

```python
[
	({
		'one': True,
		'two': False,
		'three': False,
		'four': True,
		'five': False
	}, True),
	({
		'one': False,
		'two': False,
		'three': False,
		'four': True,
		'five': True
	}, False),
	({
		'one': False,
		'two': True,
		'three': False,
		'four': False,
		'five': False
	}, True),
	({
		'one': True,
		'two': False,
		'three': False,
		'four': False,
		'five': True
	}, False)
]
```

To help you in this step, watch the [following video](https://www.youtube.com/watch?v=-vVskDsHcVc) to learn how to build the feature set with Python and NLTK. The source code in this video can be found [here](https://pythonprogramming.net/words-as-features-nltk-tutorial/).

[![Building Features](https://github.com/aaronpereira92/lab-nlp/blob/master/your-code/building-features.jpg?raw=1)](https://www.youtube.com/watch?v=-vVskDsHcVc)

In [15]:
# your code here

#First I will convert a list of tuple s
freq_words = dict(freq_words)
word_features = list(freq_words.keys())
word_features

['go',
 'get',
 'wa',
 'thi',
 'day',
 'like',
 'good',
 'quot',
 'love',
 'work',
 'u',
 'time',
 'amp',
 'know',
 'got',
 'see',
 'thank',
 'want',
 'today',
 'realli',
 'one',
 'back',
 'miss',
 'hope',
 'im',
 'think',
 'make',
 'still',
 'feel',
 'new',
 'look',
 'need',
 'much',
 'lol',
 'well',
 'ha',
 'night',
 'come',
 'oh',
 'home',
 'watch',
 'last',
 'great',
 'wait',
 'onli',
 'hi',
 'twitter',
 'wish',
 'veri',
 'sleep',
 'thing',
 'tri',
 'would',
 'morn',
 'bad',
 'tomorrow',
 'take',
 'follow',
 'sorri',
 'even',
 'way',
 'could',
 'fun',
 'whi',
 'week',
 'say',
 'right',
 'gonna',
 'let',
 'haha',
 'friend',
 'sad',
 'nice',
 'tonight',
 'yeah',
 'better',
 'start',
 'first',
 'peopl',
 'show',
 'use',
 'ye',
 'hour',
 'tweet',
 'happi',
 'cant',
 'never',
 'play',
 'next',
 'dont',
 'hey',
 'guy',
 'littl',
 'school',
 'hate',
 'call',
 'awesom',
 'long',
 'bed',
 'ur',
 'wanna',
 'movi',
 'becaus',
 'everyon',
 'befor',
 'sure',
 'alway',
 'best',
 'n',
 'though',


In [61]:
#Lets make a column is_positive to check if the sentence is positive or not
tweets['is_positive'] = tweets.target.apply(lambda x: True if x == 4 else False)

In [65]:
def find_features(document, word_features):
    words = set(document)
    features = {}
    for w in word_features:
        features[w] = (w in words)

    return features

In [71]:
featuresets = [(find_features(row.text_processed, word_features), row.is_positive) for index, row in tweets.iterrows()]


In [73]:
#Feature set for the 5000 most popular words now obtained
featuresets[0]

({'go': True,
  'get': True,
  'wa': False,
  'thi': False,
  'day': False,
  'like': False,
  'good': False,
  'quot': False,
  'love': False,
  'work': False,
  'u': False,
  'time': False,
  'amp': False,
  'know': False,
  'got': False,
  'see': False,
  'thank': False,
  'want': False,
  'today': False,
  'realli': False,
  'one': False,
  'back': False,
  'miss': False,
  'hope': False,
  'im': False,
  'think': False,
  'make': False,
  'still': False,
  'feel': False,
  'new': False,
  'look': False,
  'need': False,
  'much': False,
  'lol': False,
  'well': False,
  'ha': False,
  'night': False,
  'come': False,
  'oh': False,
  'home': False,
  'watch': False,
  'last': False,
  'great': False,
  'wait': False,
  'onli': False,
  'hi': False,
  'twitter': False,
  'wish': False,
  'veri': False,
  'sleep': False,
  'thing': False,
  'tri': False,
  'would': False,
  'morn': False,
  'bad': False,
  'tomorrow': False,
  'take': False,
  'follow': False,
  'sorri': False,
  '

### Building and Traininng Naive Bayes Model

In this step you will split your feature set into a training and a test set. Then you will create a Bayes classifier instance using `nltk.NaiveBayesClassifier.train` ([example](https://www.nltk.org/book/ch06.html)) to train with the training dataset.

After training the model, call `classifier.show_most_informative_features()` to inspect the most important features. The output will look like:

```
Most Informative Features
	    snow = True            False : True   =     34.3 : 1.0
	  easter = True            False : True   =     26.2 : 1.0
	 headach = True            False : True   =     20.9 : 1.0
	    argh = True            False : True   =     17.6 : 1.0
	unfortun = True            False : True   =     16.9 : 1.0
	    jona = True             True : False  =     16.2 : 1.0
	     ach = True            False : True   =     14.9 : 1.0
	     sad = True            False : True   =     13.0 : 1.0
	  parent = True            False : True   =     12.9 : 1.0
	  spring = True            False : True   =     12.7 : 1.0
```

The [following video](https://www.youtube.com/watch?v=rISOsUaTrO4) will help you complete this step. The source code in this video can be found [here](https://pythonprogramming.net/naive-bayes-classifier-nltk-tutorial/).

[![Building and Training NB](https://github.com/aaronpereira92/lab-nlp/blob/master/your-code/nb-model-building.jpg?raw=1)](https://www.youtube.com/watch?v=rISOsUaTrO4)

In [74]:
len(featuresets)

20000

In [75]:
# your code here

#So I'm doing a 70-30 train test split, 
#and followed the code as per the video
train_set = featuresets[:14000]
test_set = featuresets[14000:]

In [76]:
classifier = nltk.NaiveBayesClassifier.train(train_set)

Classifier accuracy percent: 71.75


In [80]:
classifier.show_most_informative_features()

Most Informative Features
                 horribl = True            False : True   =     19.1 : 1.0
                    hurt = True            False : True   =     18.2 : 1.0
                     ach = True            False : True   =     15.1 : 1.0
                     sad = True            False : True   =     15.0 : 1.0
                   tummi = True            False : True   =     14.4 : 1.0
                   worst = True            False : True   =     14.4 : 1.0
                    argh = True            False : True   =     14.4 : 1.0
                     www = True             True : False  =     13.2 : 1.0
                    lone = True            False : True   =     13.1 : 1.0
                 depress = True            False : True   =     12.4 : 1.0


### Testing Naive Bayes Model

Now we'll test our classifier with the test dataset. This is done by calling `nltk.classify.accuracy(classifier, test)`.

As mentioned in one of the tutorial videos, a Naive Bayes model is considered OK if your accuracy score is over 0.6. If your accuracy score is over 0.7, you've done a great job!

In [81]:
# your code here
print("Classifier accuracy percent:",(nltk.classify.accuracy(classifier, test_set)))

Classifier accuracy percent: 0.7175


## Bonus Question 1: Improve Model Performance

If you are still not exhausted so far and want to dig deeper, try to improve your classifier performance. There are many aspects you can dig into, for example:

* Improve stemming and lemmatization. Inspect your bag of words and the most important features. Are there any words you should furuther remove from analysis? You can append these words to further remove to the stop words list.

* Remember we only used the top 5,000 features to build model? Try using different numbers of top features. The bottom line is to use as few features as you can without compromising your model performance. The fewer features you select into your model, the faster your model is trained. Then you can use a larger sample size to improve your model accuracy score.

In [ ]:
# your code here

## Bonus Question 2: Machine Learning Pipeline

In a new Jupyter Notebook, combine all your codes into a function (or a class). Your new function will execute the complete machine learning pipeline job by receiving the dataset location and output the classifier. This will allow you to use your function to predict the sentiment of any tweet in real time. 

In [ ]:
# your code here

## Bonus Question 3: Apache Spark

If you have completed the Apache Spark advanced topic lab, what you can do is to migrate your pipeline from local to a Databricks Notebook. Share your notebook with your instructor and classmates to show off your achievements!

In [ ]:
# your code here